Copyright (C) Egon Kidmose 2015-2017

This file is part of lstm-rnn-correlation.

lstm-rnn-correlation is free software: you can redistribute it and/or
modify it under the terms of the GNU Lesser General Public License as
published by the Free Software Foundation, either version 3 of the
License, or (at your option) any later version.

lstm-rnn-correlation is distributed in the hope that it will be
useful, but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU
Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public
License along with lstm-rnn-correlation. If not, see
<http://www.gnu.org/licenses/>.


In [57]:
import pickle
import pandas as pd
import numpy as np
import sklearn

In [16]:
m_files = [
    'output/20161227-132720-js3/20161227-132720-js3-metrics.pickle',
    'output/20161227-141227-js3/20161227-141227-js3-metrics.pickle',
    'output/20161227-145922-js3/20161227-145922-js3-metrics.pickle',
    'output/20161227-154626-js3/20161227-154626-js3-metrics.pickle',
    'output/20161227-163159-js3/20161227-163159-js3-metrics.pickle',
    'output/20161227-171847-js3/20161227-171847-js3-metrics.pickle',
    'output/20161227-180341-js3/20161227-180341-js3-metrics.pickle',
    'output/20161227-185115-js3/20161227-185115-js3-metrics.pickle',
    'output/20161227-193748-js3/20161227-193748-js3-metrics.pickle',
    'output/20161227-202320-js3/20161227-202320-js3-metrics.pickle',
]

def load_pickle(filename):
    with open(filename, 'r') as f:
        return pickle.load(f)

ms = map(load_pickle, m_files)

ps = (5,2) # parameters for clustering

In [69]:
def aggregate(df):
    return pd.concat(
        [
            df.mean().rename('Mean'),
            df.min().rename('Min.'),
            df.max().rename('Max.'),
        ],
        axis=1,
    )

In [71]:
# Domain specific metrics
df_domain = pd.concat([
        pd.Series([m['imr']['validation'][ps] for m in ms], name='imr'),
        pd.Series([m['arf']['validation'][ps] for m in ms], name='arf'),
        pd.Series([m['narf']['validation'][ps] for m in ms], name='narf'),
    ], axis=1,
)
df_domain_agg = aggregate(df_domain)
df_domain_agg

,Mean,Min.,Max.
imr,0.006183,0.000000,0.017857
arf,9.247887,8.307692,9.818182
narf,0.019154,0.016955,0.020460


In [133]:
# Classification metrics
df_classify = pd.concat([
        pd.Series([m['accuracy']['validation'][ps] for m in ms], name='Accuracy'),
        pd.Series([m['precision']['validation'][ps] for m in ms], name='Precision'),
        pd.Series([m['recall']['validation'][ps] for m in ms], name='Recall'),
        pd.Series([m['f1']['validation'][ps] for m in ms], name='F1-score'),
    ], axis=1,
)
df_classify_agg = aggregate(df_classify)
df_classify_agg

,Mean,Min.,Max.
Accuracy,0.737255,0.716937,0.759259
Precision,0.731546,0.709876,0.752833
Recall,0.737255,0.716937,0.759259
F1-score,0.720425,0.702132,0.740801


In [220]:
# Classification metrics, breakdown pr. class
dfs_class_classify = [
    (pd.DataFrame(dict(m['per_class']['validation'][ps].items())), rep) 
    for rep, m in enumerate(ms)
]
df_class_classify = pd.concat(map(
    lambda (df, rep) : pd.concat([
            df,
            pd.Series([rep for x in range(len(df.index))], index=df.index, name='repetition')
        ], axis=1),
    dfs_class_classify
)).reset_index(drop=True)
df_class_classify.loc[df_class_classify['labels']==-1, 'labels'] = 'Benign'

# df_class_classify_agg = aggregate(df_class_classify[['precission', 'recall', 'f1', 'support']])
# df_class_classify_agg

df_class_classify_agg = df_class_classify[
    ['f1', 'precission', 'recall', 'support', 'labels']
].groupby(
    'labels'
).aggregate(
    ['mean', 'min', 'max']
)
df_class_classify_agg = pd.concat([
        df_class_classify_agg.reset_index(),
        df_class_classify[
            ['f1', 'precission', 'recall', 'support']
        ].groupby(
            lambda _ : 'Overall'
        ).agg(
            ['mean', 'min', 'max']
        ).reset_index().rename(columns={'index':'labels'})
]).set_index('labels')
df_class_classify_agg

f1                     precission                      \
             mean       min       max       mean       min       max   
labels                                                                 
1        0.445343  0.153846  0.700000   0.612500  0.333333  0.800000   
2        0.758291  0.551724  0.857143   0.836327  0.727273  1.000000   
3        0.985266  0.965517  1.000000   0.983826  0.935484  1.000000   
4        0.887884  0.800000  0.968750   0.891311  0.756757  1.000000   
5        0.650691  0.589474  0.720000   0.554140  0.500000  0.645833   
6        0.291355  0.218182  0.400000   0.562998  0.333333  0.875000   
7        0.450975  0.357143  0.541176   0.495887  0.357143  0.575758   
Benign   0.812151  0.786667  0.835886   0.777977  0.751037  0.803653   
Overall  0.660245  0.153846  1.000000   0.714371  0.333333  1.000000   

           recall                     support            
             mean       min       max    mean  min  max  
labels                                                   
1        0.370000  0.100000  0.700000   10.00   10   10  
2        0.700877  0.444444  0.833333   18.40   18   19  
3        0.987171  0.965517  1.000000   31.70   29   34  
4        0.893466  0.757576  1.000000   32.80   32   33  
5        0.792308  0.717949  0.923077   39.00   39   39  
6        0.202500  0.150000  0.282051   39.50   39   40  
7        0.419647  0.319149  0.534884   44.40   42   47  
Benign   0.850362  0.814815  0.906977  215.80  215  216  
Overall  0.652041  0.100000  1.000000   53.95   10  216

In [224]:
ms[0]['cm_inc_clust']['validation'][ps]

,noise,0,1,2,3,4,5,6,7,8,...,39,40,41,42,43,44,45,46,47,49
benign,29,56,11,4,1,0,2,0,2,1,...,4,7,0,1,0,1,1,1,0,1
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,29,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,15,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0
5,2,0,0,25,3,0,0,0,1,0,...,0,0,1,0,0,0,2,0,0,0
6,12,0,0,10,1,0,0,2,0,0,...,0,0,0,0,0,0,1,0,1,0
7,22,0,0,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [225]:
ms[0]['cm_inc_inc']['validation'][ps]

,benign,0,1,2,3,4,5,6,7,8,...,39,40,41,42,43,44,45,46,47,49
benign,29,56,11,4,1,0,2,0,2,1,...,4,7,0,1,0,1,1,1,0,1
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,29,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,15,0,0,0,0,...,0,0,0,2,1,0,0,0,0,0
5,2,0,0,25,3,0,0,0,1,0,...,0,0,1,0,0,0,2,0,0,0
6,12,0,0,10,1,0,0,2,0,0,...,0,0,0,0,0,0,1,0,1,0
7,22,0,0,0,0,2,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
